In [1]:
import pyforest
from rdkit import Chem
from rdkit.Chem import AllChem
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE 

In [3]:
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'

In [15]:
df = pd.read_csv(datapath+'final_pIC50s_for_ML_noOut_noDupes_pivot.csv')
print(df.shape)
df=df.dropna()
print(df.shape)

<IPython.core.display.Javascript object>

(400, 10)
(400, 10)


In [18]:
df['smiles'].unique().shape

(400,)

In [33]:
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/original_datasets/'
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition']
all_smiles_lists = []
for nek in [2, 3, 5, 9]: 
    nek_type_list = ['binding']
    if nek in [2, 9]: 
        nek_type_list =['binding', 'inhibition']
    for bi in nek_type_list: 
        nek_df = pd.read_csv(f'{og_datapath}NEK{nek}_1_uM_min_50_pct_{bi}.csv')
        all_smiles_lists.append(nek_df['base_rdkit_smiles'].values)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
all_smiles = []
for sublist in all_smiles_lists:
    all_smiles.extend(sublist)
len(all_smiles)

In [55]:
mol_in_og_data = df[df['smiles'].isin(all_smiles)]
new_df = df[~df['smiles'].isin(all_smiles)]
mol_in_og_data
new_df
print(f'new nanobret data size after filtering: {new_df.shape}')
for nek in [ x for x in new_df.columns if x.startswith('NEK')]: 
    new_df[f'{nek}_active']=new_df[nek].apply(lambda x: 1 if x>6 else 0)

new nanobret data size after filtering: (400, 10)


In [61]:
new_df

,Compound,smiles,NEK1,NEK11,NEK2,NEK3,NEK4,NEK5,NEK6,NEK9,NEK1_active,NEK11_active,NEK2_active,NEK3_active,NEK4_active,NEK5_active,NEK6_active,NEK9_active
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,4.522879,4.741265,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,0,0,0,0,0,0,0,0
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,4.522879,4.522879,4.536830,4.522879,4.543123,4.522879,4.522879,4.522879,0,0,0,0,0,0,0,0
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,0,0,0,0,0,0,0,0
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,4.522879,4.772799,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,0,0,0,0,0,0,0,0
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,5.338173,4.522879,4.522879,6.231165,4.522879,6.334277,4.522879,5.329514,0,0,0,1,0,1,0,0
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,4.956505,4.701115,4.990639,4.522879,4.746844,6.067683,4.522879,5.269689,0,0,0,0,0,1,0,0
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,5.021631,4.987176,4.920219,4.522879,5.030679,4.522879,4.522879,4.771226,0,0,0,0,0,0,0,0
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,4.522879,4.522879,4.522879,4.522879,4.522879,4.880795,4.522879,5.908011,0,0,0,0,0,0,0,0


In [70]:
for nek in [ x for x in new_df.columns if x.endswith('active')]: 
    nek_str = nek.split('_')[0]

    cols = ['Compound', 'smiles', nek] 
    nek_df = new_df[cols]
    # display(nek_df)
    display(nek_df[nek].value_counts())
    nek_df.to_csv(f'{datapath}{nek_str}_uM_min_50_pct.csv', index=False)

NEK1_active
0    370
1     30
Name: count, dtype: int64

NEK11_active
0    398
1      2
Name: count, dtype: int64

NEK2_active
0    369
1     31
Name: count, dtype: int64

NEK3_active
0    372
1     28
Name: count, dtype: int64

NEK4_active
0    400
Name: count, dtype: int64

NEK5_active
0    320
1     80
Name: count, dtype: int64

NEK6_active
0    400
Name: count, dtype: int64

NEK9_active
0    366
1     34
Name: count, dtype: int64

# Next steps
1. figure out binding or inhibition data --> pIC50 of 6 is equivalent to 50% inhibition at 1uM
2. how should we convert the labels to active/inactive? (our previous datasets were perecent binding/inhibition)
4. run this new data on our production models 